## GBDT算法梳理

1. 前向分布算法
- 负梯度拟合
- 损失函数
- 回归
- 二分类，多分类
- 正则化
- 优缺点
- sklearn参数
- 应用场景

GBDT是一种采用加法模型（基函数的线性组合）与前向分布算法并以决策树作为函数的提升方法；该算法由多颗决策树组成，所有树的结论加起来形成最终答案；

### 前向分布算法

加法模型是这样的：$$f(x)=\sum_{m=1}^{M}\beta_mb(x;\gamma_m)$$；(就是基学习器的一种线性组合)
其中，$b(x;\gamma_m)$为基函数，$\gamma_m$为基函数的参数，$\beta_m$为基函数的系数。
在给定训练数据及损失函数$L(y,f(x))$的条件下，学习加法模型成为损失函数极小化问题：
$$
min_{\beta_m,\gamma_m}\sum_{i-1}^{N}L(y_i,\sum_{m=1}^{M}\beta_mb(x_i;\gamma_m))
$$
前向分布算法求解这一优化问题的思路：因为学习的是加法模型，如果能够从前向后，每一步只学习一个基函数及其系数，逐步去逼近上述的目标函数式，就可以简化优化的复杂度，每一步秩序优化如下损失函数：
$$
min_\beta,\gamma \sum_{i=1}^{N}L(y_i,\beta b(x;\gamma))
$$（每步学习一个基函数和系数）

### 负梯度拟合

提升方法实际采用加法模型和前向分布 算法，以决策树作为基函数的提升方法；决策树称为CART回归树；

### 损失函数
1. 对于分类算法，其损失函数一般由对数损失函数和指数损失函数两种。
 - 指数损失函数表达式：
 $$
 L(y,f(x))=e^{-yf(x)}
 $$
 - 对数损失函数可分为二分类和多分类两种。
2. 对于回归算法，常用损失函数有如下4种。
 - 平方损失函数：
 $$
 L(y,f(x))=(y-f(x))^2
 $$
 - 绝对损失函数：
 $$
 L(y,f(x))=|y-f(x)|
 $$
 对应负梯度误差为：$sign(y_i-f(x_i))$
 - Huber损失，他说均方差和绝对损失的折中产物，对于远离中心的异常点，采用绝对损失误差，而对于靠近中心的点则采用平方损失。这个界限一般用分位数点度量；损失函数如下：
 $$
L(y,f(x)))=\left\{\begin{matrix}
\frac{1}{2}(y-f(x))^2 & |y-f(x)|\leq \delta \\ 
\delta(|y-f(x)|-\frac{\delta}{2}) & |y-f(x)|>  \delta \\
\end{matrix}\right.
 $$
 对应的负梯度误差为：
 $$
 r(y_i,f(x_i)))=\left\{\begin{matrix}
y_i-f(x_i)& |y_i-f(x_i)|\leq \delta \\
\delta sign(y_i-f(x_i))&|y_i-f(x_i)|> \delta \\
\end{matrix}\right.
$$
 - 分位数损失。它对应的是分位数回归的损失函数，表达式为：
$$
 L(y,f(x))=\sum_{y\geq f(x)}\theta|y-f(x)|+\sum_{y<f(x)}(1-\theta)|y-f(x)|
$$
其中$\theta$为分位数，需要我们在回归之前指定。对应的负梯度误差为：
$$
 r(y_i,f(x_i)))=\left\{\begin{matrix}
\theta & y_i\geq f(x_{i}) \\
\theta-1 & y_i> f(x_{i}) \\
\end{matrix}\right.
$$

### 回归问题
梯度提升算法（回归问题）流程(P.151)：
输入：训练数据集$T={(x_1,y_1),(x_2,y_2),...,(x_N,y_N)},x_i \in \chi \subseteq R^n,y_i\in\gamma\subseteq R$;损失函数$L(y,f(x))$；

输出：回归树$\tilde{f(x)}$;

 - 初始化
 $f_0=argmin_c\sum_{i=1}^{N}L(y_i,c)$，注：估计使损失函数极小化的常数值，它是只有一个根节点的树
 - 对$m=1,2,...M$
  - 对m=1,2,...,N,计算
  $$
  r_mi=-\begin{bmatrix}
\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}
\end{bmatrix} _{f(x)=f_m-1(x)}
  $$ 注：计算损失函数在当前模型的值，作为残差的估计
  - 对$r_mi$拟合一个回归树，得到第$m$棵树的叶结点区域$R_my$，$j=1,2,...,J$
  - 对$j=1,2,...,J$，计算
  $$
  c_{mj}=argmin_c\sum_{x_i\in R_mj}L(y_i,f_{m-1}(x_i)+c)
  $$**注：在损失函数极小化条件下，估计出相应叶结点区域的值**
  - 更新$f_m(x)=f_m-1(x)+\sum_{j=1}^{J}c_{mj}I(x\in  R_{mj})$
 - 得到回归树
 $$
 \tilde{f(x)}=f_M(x)=\sum_{m-1}^M \sum_{j=1}^J c_{mj}I(x\in R_{mj})
 $$
 

### 分类问题（二分类与多分类）
这里看看GBDT分类算法，GBDT的分类算法从思想上和GBDT的回归算法没有区别，但是由于样本输出不是连续的值，而是离散的类别，导致我们无法直接从输出类别去拟合输出类别的误差。

为了解决这个问题，主要有两个方法，一个是用指数损失函数，此时GBDT退化为Adaboost算法。另一种方法用类似逻辑回归的对数似然函数的方法。也就是说，我们用的是类别的预测概率值和真实概率值的差来拟合损失。此处仅讨论用对数似然函数的GBDT分类。对于对数似然损失函数，我们有又有二元分类和的多元分类的区别。

1. 二分类GBDT算法

对于二分类GBDT，如果用类似逻辑回归的对数似然损失函数，则损失函数为：
$L(y,f(x))=log(1+exp(-yf(x)))$
其中$y\in{-1,1}$。此时的负梯度误差为：
$$
r_ti=-\begin{bmatrix}
\frac{\partial L(y,f(x_i))}{\partial f(x_i)}
\end{bmatrix}_{f(x)=f_{t-1}(x)} = \frac {y_i}{1-exp(y_if(x_i))}
$$
对于生成的决策树，我们各个叶子节点的最佳负梯度拟合值为
$$
c_tj=argmin_c \sum {x_i \in R_{tj}}log(1+exp(-y_i(f_{t-1}(x_i)+c)))
$$
由于上式比较难优化，我们一般使用近似值代替：
$$
c_{tj}=\frac{\sum_{ x_i \in R_{ij} }r_{tj}}{\sum_{x_i \in R_{ij}}|r_{ij}|(1-|r_{ij}|)}
$$
除了负梯度计算和叶子节点的最佳负梯度拟合的线性搜索，二分类GBDT与GBDT回归算法过程相同。
2. 多分类GBDT算法

多分类GBDT比二分类GBDT复杂些，对应的是多元逻辑回归和二元逻辑回归的复杂度差别。假设类别数为K，则此时我们的对数似然损失函数为：
$$
L(y,f(x))=-\sum_{k=1}^{K}y_k log(p_k(x))
$$
其中如果样本输出类别为k,则$y_k=1$，第$k$类的概率$p_k(x)$的表达式为：
$$
p_x(x)=\frac {exp(f_x(x))} {\sum_{i-1}^{K}exp(f_i(x))}
$$
集合上两式，我们可以计算出第$t$轮的第$i$个样本对应类别的负梯度误差为：
$$
t_{til}=-\begin{bmatrix}
\frac {\partial L(y_i,f(x_i))}{\partial f(x_i)}
\end{bmatrix} _{f_k(x)=f_{l,t-1}(x)} = y_{il}-p_{l,t-1}(x_i)
$$
观察上式可以看出，其实这里的误差就是样本$i$对应类别$l$的真是概率和$t-1$轮预测概率的差值。

对于生成的决策树，我们各个叶子节点的最佳负梯度拟合值为：
$$
c_{tjl}=argmin_{c_{jl}}\sum_{i=0}^{m} \sum_{k=1}^{K}L(y_k,f_{t-1,l}(x)+\sum_{j=0}^{J}c_{jl}I(x_i \in R_{tj}))
$$
由于上式比较难优化，我们一般使用近似值代替：
$$
c_{tjl}=\frac {K-1}{K}\frac {\sum_{x_i \in R_{tjl}}r_{til}} {\sum_{x_i \in R_{til}}|r_{til}|(1-|r_{til}|)}
$$
除了负梯度计算和叶子节点的最佳负梯度拟合的线性搜索，多分类GBDT与二分类GBDT以及GBDT回归算法过程相同。

### 正则化
对GBDT进行正则化来防止过拟合，主要有三种形式。
1. 给每棵数的输出结果乘上一个步长$a$（learning rate）。

对于前面的弱学习器的迭代：
$$
f_m(x)=f_{m-1}(x)+T(x;\gamma_m)
$$
加上正则化项，则有
$$
f_m(x)=f_{m-1}(x)+aT(x;\gamma_m)
$$
此处，$a$的取值范围为(0,1]。对于同样的训练集学习效果，较小的$a$意味着需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起决定算法的拟合效果。

2. 第二种正则化的方式就是通过子采样比例(subsample)。取值范围为(0,1]。

GBDT这里的做法是在每一轮建树时，样本是从原始训练集中采用无放回随机抽样的方式产生，与随机森立的有放回抽样产生采样集的方式不同。若取值为1，则采用全部样本进行训练，若取值小于1，则不选取全部样本进行训练。选择小于1的比例可以减少方差，防止过拟合，但可能会增加样本拟合的偏差。取值要适中，推荐[0.5,0.8]。

3.第三种是对弱学习器即CART回归树进行正则化剪枝。（如控制树的最大深度、节点的最少样本数、最大叶子节点数、节点分支的最小样本数等）

### GBDT优缺点
1. GBDT优点

可以灵活处理各种类型的数据，包括连续值和离散值。
在相对较少的调参时间情况下，预测的准确率也比较高，相对SVM而言。
在使用一些健壮的损失函数，对异常值得鲁棒性非常强。比如Huber损失函数和Quantile损失函数。
2. GBDT缺点

由于弱学习器之间存在较强依赖关系，难以并行训练。可以通过自采样的SGBT来达到部分并行。

### sklearn参数
在scikit-learning中，GradientBoostingClassifier对应GBDT的分类算法，GradientBoostingRegressor对应GBDT的回归算法。

具体算法参数情况如下：
```python
GradientBoostingRegressor(loss=’ls’, learning_rate=0.1, n_estimators=100, 
                subsample=1.0, criterion=’friedman_mse’, min_samples_split=2,
                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3,
                min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                random_state=None, max_features=None, alpha=0.9, verbose=0, 
                max_leaf_nodes=None, warm_start=False, presort=’auto’, 
                validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)
```

**参数说明：**

- n_estimators：弱学习器的最大迭代次数，也就是最大弱学习器的个数。
- learning_rate：步长，即每个学习器的权重缩减系数a，属于GBDT正则化方化手段之一。
- subsample：子采样，取值(0,1]。决定是否对原始数据集进行采样以及采样的比例，也是GBDT正则化手段之一。
- init：我们初始化的时候的弱学习器。若不设置，则使用默认的。
- loss：损失函数，可选{'ls'-平方损失函数，'lad'绝对损失函数-,'huber'-huber损失函数,'quantile'-分位数损失函数}，默认'ls'。
- alpha：当我们在使用Huber损失"Huber"和分位数损失"quantile"时，需要指定相应的值。默认是0.9，若噪声点比较多，可适当降低这个分位数值。
- criterion：决策树节搜索最优分割点的准则，默认是"friedman_mse"，可选"mse"-均方误差与'mae"-绝对误差。
- max_features：划分时考虑的最大特征数，就是特征抽样的意思，默认考虑全部特征。
- max_depth：树的最大深度。
- min_samples_split：内部节点再划分所需最小样本数。
- min_samples_leaf：叶子节点最少样本数。
- max_leaf_nodes：最大叶子节点数。
- min_impurity_split：节点划分最小不纯度。
- presort：是否预先对数据进行排序以加快最优分割点搜索的速度。默认是预先排序，若是稀疏数据，则不会预先排序，另外，稀疏数据不能设置为True。
- validationfraction：为提前停止而预留的验证数据比例。当n_iter_no_change设置时才能用。
- n_iter_no_change：当验证分数没有提高时，用于决定是否使用早期停止来终止训练。



### GBDT应用场景
GBDT几乎可以用于所有回归问题（线性/非线性），相对loigstic regression仅能用于线性回归，GBDT的适用面非常广。亦可用于分类问题。